**TP1 -Analyse vidéo**

Benzaied Saifeddine -RT5 groupe 1

I. Acquisition Chargement et Sauvegarde

In [1]:
import cv2

# Chargement de la vidéo
video = cv2.VideoCapture("my_video_TPsAV.mp4")

# Vérification si la vidéo a bien été chargée
if not video.isOpened():
    print("Erreur lors de l'ouverture de la vidéo")
else:
    print("Vidéo chargée avec succès")


Vidéo chargée avec succès


In [2]:
# Récupérer les informations de la vidéo
hauteur = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
largeur = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(video.get(cv2.CAP_PROP_FPS))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Hauteur: {hauteur}, Largeur: {largeur}, FPS: {fps}, Nombre de frames: {total_frames}")


Hauteur: 720, Largeur: 1280, FPS: 15, Nombre de frames: 164


In [3]:
nbFrame = 0
while True:
    ret, frame = video.read()  # Lire la prochaine frame
    if not ret:
        break  # Si aucune frame n'est retournée, on sort de la boucle

    if nbFrame % 30 == 0:  # Sauvegarde de chaque 30 frame
        frame_name = f"frame_{nbFrame}.png"
        cv2.imwrite(frame_name, frame)  # Sauvegarde de la frame dans un fichier image
        print(f"Sauvegarde de {frame_name}")

    nbFrame += 1
    
video.release()

Sauvegarde de frame_0.png
Sauvegarde de frame_30.png
Sauvegarde de frame_60.png
Sauvegarde de frame_90.png
Sauvegarde de frame_120.png
Sauvegarde de frame_150.png


In [4]:
# Chargement de la vidéo
video = cv2.VideoCapture("my_video_TPsAV.mp4")

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    cv2.imshow("Frame", frame)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

# Fermer toutes les fenêtres
cv2.destroyAllWindows()
video.release()

In [12]:
video.release()
cv2.destroyAllWindows()

II. Segmentation spatiale: Conversion et seuillage

In [13]:
video = cv2.VideoCapture("my_video_TPsAV.mp4")

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Conversion en niveaux de gris
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Afficher l'image en niveaux de gris
    cv2.imshow("Gray Frame", gray_frame)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
video.release()


In [14]:
import numpy as np

def seuillage_heuristique(gray_frame, T_initial):
    T = T_initial
    while True:
        # Segmentation avec le seuil actuel
        _, img_seuil = cv2.threshold(gray_frame, T, 255, cv2.THRESH_BINARY)

        # Pixels plus clairs que le seuil
        above_T = gray_frame[gray_frame > T]
        # Pixels plus sombres ou égaux au seuil
        below_T = gray_frame[gray_frame <= T]

        # Calcul de la nouvelle valeur du seuil avec vérification pour éviter les tranches vides
        mean_above_T = above_T.mean() if above_T.size > 0 else T
        mean_below_T = below_T.mean() if below_T.size > 0 else T

        T_new = (mean_above_T + mean_below_T) / 2

        if abs(T - T_new) < 1:  # Convergence
            break
        T = T_new

    return img_seuil, T

video = cv2.VideoCapture("my_video_TPsAV.mp4")

# Obtenir le nombre total de frames dans la vidéo
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Nombre total de frames : {total_frames}")

middle_frame_idx = total_frames // 2

# Se positionner à la frame du milieu
video.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_idx)

# Lire la frame
ret, frame = video.read()

if ret:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Appliquer le seuillage
    img_seuil, T_final = seuillage_heuristique(gray_frame, 150)

    cv2.imshow("Image suillée", img_seuil)
    print("Seuil final: ",T_final)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Erreur lors du chargement de la vidéo")


Nombre total de frames : 164
Seuil final:  116.76290688326392


In [15]:
while True:
    ret, frame = video.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Appliquer le seuillage d'Otsu
    _, img_otsu = cv2.threshold(gray_frame, 50, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Afficher la vidéo d'origine et la vidéo segmentée
    cv2.imshow("Original Video", frame)
    cv2.imshow("Otsu Segmented Video", img_otsu)

    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
video.release()


III. Segmentation temporelle

In [16]:
import numpy as np

# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')

# Initialiser la première frame comme image de fond
ret, frame_prev = video.read()
frame_prev_gray = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convertir la frame actuelle en niveaux de gris
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Soustraction de l'image précédente
    diff = cv2.absdiff(frame_gray, frame_prev_gray)

    # Seuillage
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Afficher la différence et l'image seuillée
    cv2.imshow("Frame Actuelle", frame)
    cv2.imshow("Soustraction d images", diff)
    cv2.imshow("Seuillage", thresh)

    # Mettre à jour la frame précédente
    frame_prev_gray = frame_gray

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [18]:
import cv2

# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')

# Nombre total de frames dans la vidéo
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Demander à l'utilisateur de choisir la position de la frame
frame_position = int(input(f"Entrez la position de la frame à afficher (entre 0 et {total_frames-1}): "))

# Se déplacer jusqu'à la frame choisie
video.set(cv2.CAP_PROP_POS_FRAMES, frame_position)

# Lire la frame choisie et la frame précédente pour la soustraction
ret, frame_prev = video.read()
frame_prev_gray = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)

# Lire la frame actuelle (celle que l'on veut afficher)
ret, frame = video.read()
if ret:
    # Convertir la frame actuelle en niveaux de gris
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Soustraction de la frame précédente
    diff = cv2.absdiff(frame_gray, frame_prev_gray)

    # Appliquer le seuillage
    _, thresh = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)

    # Afficher les résultats
    cv2.imshow("Frame Originale", frame)  # Frame originale
    cv2.imshow("Soustraction", diff)      # Résultat de la soustraction
    cv2.imshow("Seuillage", thresh)       # Résultat du seuillage

    # Attendre jusqu'à ce que l'utilisateur appuie sur une touche pour fermer les fenêtres
    cv2.waitKey(0)

# Libérer la vidéo et fermer toutes les fenêtres
video.release()
cv2.destroyAllWindows()


Entrez la position de la frame à afficher (entre 0 et 163): 100


In [19]:
# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')

# Choix de N (décalage de N frames)
N = 5

# Lire les frames initiales pour la comparaison
ret, frame_prev = video.read()
for i in range(N):  # Sauter N frames avant de lire la frame actuelle
    ret, frame_curr = video.read()
ret, frame_next = video.read()

while True:
    # Convertir les frames en niveaux de gris
    prev_gray = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(frame_curr, cv2.COLOR_BGR2GRAY)
    next_gray = cv2.cvtColor(frame_next, cv2.COLOR_BGR2GRAY)

    # Calcul des différences entre la frame actuelle et la frame décalée de N frames
    diff1 = cv2.absdiff(prev_gray, curr_gray)
    diff2 = cv2.absdiff(curr_gray, next_gray)

    # Opération OR pour combiner les zones de mouvement
    motion = cv2.bitwise_or(diff1, diff2)

    # Appliquer un flou pour réduire le bruit
    motion_blur = cv2.GaussianBlur(motion, (5, 5), 0)

    # Appliquer le seuillage
    _, motion_thresh = cv2.threshold(motion_blur, 20, 255, cv2.THRESH_BINARY)

    # Afficher le résultat
    cv2.imshow("Détection de mouvement", motion_thresh)

    # Lire la prochaine frame et mettre à jour, sauter N frames pour la comparaison
    frame_prev = frame_curr
    for i in range(N):  # Sauter N frames pour la comparaison suivante
        ret, frame_curr = video.read()
    ret, frame_next = video.read()
    
    if not ret:
        break

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [20]:
alpha = 0.45

# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')

# Lire la première frame pour initialiser l'image de fond
ret, frame = video.read()
background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convertir la frame actuelle en niveaux de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculer la différence absolue
    diff = cv2.absdiff(background.astype(np.uint8), gray)

    # Seuillage
    _, motion_mask = cv2.threshold(diff, 10, 255, cv2.THRESH_BINARY)

    # Afficher la détection de mouvement
    cv2.imshow("Détection de mouvement", motion_mask)

    # Mise à jour du fond avec la méthode Adaptive Background Subtraction
    background = alpha * gray + (1 - alpha) * background

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

In [21]:
# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')

# Création des soustracteurs de fond
meth_mog2 = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)
meth_knn = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=1000)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Application de la soustraction de fond avec MOG2
    mask_mog2 = meth_mog2.apply(frame)

    # Application de la soustraction de fond avec KNN
    mask_knn = meth_knn.apply(frame)

    # Affichage des résultats
    cv2.imshow("MOG2", mask_mog2)
    cv2.imshow("KNN", mask_knn)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
